In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
from pandas_profiling import ProfileReport as pp
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from collections import Counter


import warnings
warnings.filterwarnings("ignore")
sns.set(style="ticks", context="talk")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')
data.head()

In [ ]:
data.info()

In [ ]:
pp(data)

In [ ]:
def bar(var):
    cnt = data[var].value_counts().reset_index()
# cnt.reset_index()
    cnt.plot.bar(x='index', y=var, grid=True, legend=False, title=var)


In [ ]:
categorical = (data.dtypes == "object")
categorical_list = list(categorical[categorical].index)

# print("Categorical variables:")
# print(categorical_list)
sns.set_style('darkgrid')

for c in categorical_list:
    bar(c)

In [ ]:
data.drop('id',inplace=True, axis=1)


In [ ]:
data.dtypes

In [ ]:
# ht_cnt = data['hypertension'].value_counts().reset_index() 
# print(ht_cnt)
# plt.pie(x=ht_cnt['hypertension'], labels=['No', 'Yes'])

In [ ]:
plt.figure(figsize=(25,15))

plt.subplot(2,3,1)
sns.histplot(data['age'], kde=True).set_title('Age Intervals Count')

plt.subplot(2,3,2)
sns.histplot(data['avg_glucose_level'], kde=True).set_title('Glucose Level Intervals Count')

plt.subplot(2,3,3)
sns.histplot(data['bmi'], kde=True).set_title('BMI Intervals Count')

In [ ]:
plt.figure(figsize=(25,15))


plt.subplot(2,3,1)
sns.boxenplot(x=data['stroke'], y=data['age'],
              color="b", 
              scale="linear", data=data)

plt.subplot(2,3,2)
sns.boxenplot(x=data['stroke'], y=data['bmi'],
              color="b", 
              scale="linear", data=data)

plt.subplot(2,3,3)
sns.boxenplot(x=data['stroke'], y=data['avg_glucose_level'],
              color="b", 
              scale="linear", data=data)
# plt.subplot(2,3,2)
# sns.boxenplot(x=data['stroke'], y=data['age'],
#               color="b", 
#               scale="linear", data=data)

# plt.subplot(2,3,3)
# sns.swarmplot(x="stroke", y="age",hue="smoking_status", data=data, palette="PRGn")



In [ ]:

plt.figure(figsize=(12,8)) 
sns.heatmap(data.corr())
plt.show()

In [ ]:
print(data.shape)
data.dropna(inplace=True)
print(data.shape)
# data.isnull().sum()

data.gender.value_counts()

In [ ]:
le = LabelEncoder()
data['gender'] = le.fit_transform(data['gender'])
data['ever_married'] = le.fit_transform(data['ever_married'])
data['work_type'] = le.fit_transform(data['work_type'])
data['Residence_type'] = le.fit_transform(data['Residence_type'])
data['smoking_status'] = le.fit_transform(data['smoking_status'])
data.head()
data.dtypes

In [ ]:

def detect_outliers(df,features):
    outlier_indices = []
    
    for c in features:
        # 1st quartile
        Q1 = np.percentile(df[c],25)
        # 3rd quartile
        Q3 = np.percentile(df[c],75)
        # IQR
        IQR = Q3 - Q1
        # Outlier step
        outlier_step = IQR * 1.5
        # detect outlier and their indeces
        outlier_list_col = df[(df[c] < Q1 - outlier_step) | (df[c] > Q3 + outlier_step)].index
        # store indeces
        outlier_indices.extend(outlier_list_col)
    
    outlier_indices = Counter(outlier_indices)
    multiple_outliers = list(i for i, v in outlier_indices.items() if v > 2)
    
    return multiple_outliers
data = data.drop(detect_outliers(data, ['age', 'avg_glucose_level', 'bmi', 'hypertension', 'heart_disease', 'stroke']), axis=0).reset_index(drop=True)

In [ ]:
data.head()

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# creating instance of one-hot-encoder
enc = OneHotEncoder(handle_unknown='ignore')
# passing bridge-types-cat column (label encoded values of bridge_types)
enc_df = pd.DataFrame(enc.fit_transform(data[['hypertension']]).toarray())
enc_df=enc_df.rename(columns={0 : 'hypertension_n', 1:'hypertension_y' })
data = data.join(enc_df)

enc_df = pd.DataFrame(enc.fit_transform(data[['smoking_status']]).toarray())
enc_df=enc_df.rename(columns={0 : 'smoking_status_0', 1:'smoking_status_1', 2:'smoking_status_2', 3:'smoking_status_3' })
data = data.join(enc_df)

enc_df = pd.DataFrame(enc.fit_transform(data[['Residence_type']]).toarray())
enc_df=enc_df.rename(columns={0 : 'Residence_type_n', 1:'Residence_type_y' })
data = data.join(enc_df)

enc_df = pd.DataFrame(enc.fit_transform(data[['work_type']]).toarray())
enc_df=enc_df.rename(columns={0 : 'work_type_n', 1:'work_type_y', 2:'work_type_2', 3:'work_type_3', 4:'work_type_4' })
data = data.join(enc_df)

enc_df = pd.DataFrame(enc.fit_transform(data[['ever_married']]).toarray())
enc_df=enc_df.rename(columns={0 : 'ever_married_n', 1:'ever_married_y' })
data = data.join(enc_df)

enc_df = pd.DataFrame(enc.fit_transform(data[['heart_disease']]).toarray())
enc_df=enc_df.rename(columns={0 : 'heart_disease_n', 1:'heart_disease_y' })
data = data.join(enc_df)

data.drop(['hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'smoking_status'], axis=1, inplace=True)

data.head()

In [ ]:
Y = data['stroke']
X = data.drop(['stroke'], axis=1)
X.head()

In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 2)
X_sm, Y_sm = sm.fit_resample(X, Y)
Y_sm.value_counts()
  

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import CategoricalNB
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier

from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier, XGBRFClassifier
from xgboost import plot_tree, plot_importance
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split


In [ ]:
sc=StandardScaler()

X_sm = sc.fit_transform(X_sm)
X_train, X_test, Y_train, Y_test = train_test_split(X_sm, Y_sm, test_size=0.6, random_state=101) 

X_test, X_valid, Y_test, Y_valid = train_test_split(X_test, Y_test, test_size=0.5, random_state=101) 


In [ ]:
models = {
    'GaussianNB': GaussianNB(),
    'BernoulliNB': BernoulliNB(),
    'LogisticRegression': LogisticRegression(),
    'RandomForestClassifier': RandomForestClassifier(),
    'SupportVectorMachine': SVC(),
    'DecisionTreeClassifier': DecisionTreeClassifier(),
    'KNeighborsClassifier': KNeighborsClassifier(),
    'GradientBoostingClassifier': GradientBoostingClassifier(),
    'Stochastic Gradient Descent':  SGDClassifier(max_iter=5000, random_state=0),
#     'Neural Nets': MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5000, 10), random_state=1),
}

modelNames = models.keys()

trainScores = []
validationScores = []
testScores = []

for m in models:
  model = models[m]
  model.fit(X_train, Y_train)
  score = model.score(X_valid, Y_valid)
  print(f'{m} validation score => {score*100}')
    
  print(f'{m}') 
  train_score = model.score(X_train, Y_train)
  print(f'Train score of trained model: {train_score*100}')
  trainScores.append(train_score*100)

  validation_score = model.score(X_valid, Y_valid)
  print(f'Validation score of trained model: {validation_score*100}')
  validationScores.append(validation_score*100)

  test_score = model.score(X_test, Y_test)
  print(f'Test score of trained model: {test_score*100}')
  testScores.append(test_score*100)
  print(" ")
    
  y_predictions = model.predict(X_test)
  conf_matrix = confusion_matrix(y_predictions, Y_test)

  print(f'Confussion Matrix: \n{conf_matrix}\n')

  predictions = model.predict(X_test)
  cm = confusion_matrix(predictions, Y_test)

  tn = conf_matrix[0,0]
  fp = conf_matrix[0,1]
  tp = conf_matrix[1,1]
  fn = conf_matrix[1,0]
  accuracy  = (tp + tn) / (tp + fp + tn + fn)
  precision = tp / (tp + fp)
  recall    = tp / (tp + fn)
  f1score  = 2 * precision * recall / (precision + recall)
  specificity = tn / (tn + fp)
  print(f'Accuracy : {accuracy}')
  print(f'Precision: {precision}')
  print(f'Recall   : {recall}')
  print(f'F1 score : {f1score}')
  print(f'Specificity : {specificity}')
  print("") 
  print(f'Classification Report: \n{classification_report(predictions, Y_test)}\n')
  print("")
   
#   modelNames.remove(modelNames[0])

  preds = model.predict(X_test)
  confusion_matr = confusion_matrix(Y_test, preds) #normalize = 'true'
  print("############################################################################")
  print("")
  print("")
  print("")

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential ([
#     layers.Dense(64, activation='relu', input_shape=(X_sm.shape[-1],)),
#     layers.Dense(128, activation='relu'),
#     layers.Dense(256, activation='relu'),
#     layers.Dropout(0.3),
#     layers.Dense(256, activation='relu'),
#     layers.Dropout(0.3),
#     layers.Dense(1, activation='sigmoid'),
        keras.layers.Dense(64,input_shape=(X_sm.shape[-1],),activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64,activation='relu'),
    keras.layers.Dense(2, activation='softmax')
    
])


In [ ]:

# X_test = sc.transform(X_test)

In [ ]:
metrics = [
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-2), loss="sparse_categorical_crossentropy", metrics=['accuracy']
)

callbacks = [keras.callbacks.ModelCheckpoint("stroke_model_at_epoch_{epoch}.h5")]

model.fit(
    X_sm,
    Y_sm,
#     batch_size=2048,
    epochs=30,
#     verbose=2,
    validation_split=0.25,
    callbacks=callbacks,
)